In [6]:
import scipy
import numpy as np
import pandas as pd
import seaborn as sns
import seaborn.objects as so

from sklearn import linear_model    # Herramientas de modelos lineales
from sklearn.metrics import mean_squared_error, r2_score    # Medidas de desempeño
from sklearn.preprocessing import PolynomialFeatures    # Herramientas de polinomios

from sklearn.model_selection import train_test_split, KFold, cross_val_score

from formulaic import Formula

# Práctica 5: Modelo Lineal Multivariado y K-folding

In [7]:
penguins = sns.load_dataset("penguins")
penguins.head()

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,Male
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,Female
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,Female
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,Female


In [8]:
penguins.isnull().values.any()

True

In [11]:
df = penguins.dropna()
df.isnull().values.any()

False

In [15]:
df_train, df_test = train_test_split(df, test_size=0.2, random_state=9, shuffle=True)
df_train.shape, df_test.shape

((266, 7), (67, 7))

In [27]:
y, X = (
    Formula("bill_length_mm	~ bill_depth_mm + flipper_length_mm + body_mass_g + sex")
    .get_model_matrix(df_train)
)
display(X,y)

,Intercept,bill_depth_mm,flipper_length_mm,body_mass_g,sex[T.Male]
265,1.0,13.9,217.0,4900.0,0
303,1.0,15.9,224.0,5350.0,1
20,1.0,18.3,174.0,3400.0,0
326,1.0,14.7,210.0,4700.0,0
165,1.0,18.1,201.0,4050.0,1
...,...,...,...,...,...
71,1.0,18.4,190.0,3900.0,1
131,1.0,19.2,197.0,3500.0,1
258,1.0,13.6,208.0,4350.0,0
62,1.0,17.0,185.0,3600.0,0


,bill_length_mm
265,43.6
303,50.0
20,37.8
326,41.7
165,52.0
...,...
71,39.7
131,43.1
258,44.0
62,37.6


In [34]:
modelo = linear_model.LinearRegression(fit_intercept=False)

In [31]:
cv = KFold(n_splits=5, shuffle= True, random_state=9)
pliegos = cv.split(df_train)


In [32]:
res = []
for train_index, val_index in pliegos:
    X_train, X_val, y_train, y_val = X.iloc[train_index], X.iloc[val_index], y.iloc[train_index], y.iloc[val_index]
    modelo.fit(X_train, y_train)
    y_pred = modelo.predict(X_val)
    res.append(np.sqrt(mean_squared_error(y_pred, y_val)))
res # El promedio de esta lista, es el valor a minimizar, una vez hecho eso, calculo nuevos coef sobre todo df_test, 
    # devuelvo el valor evaluado en df_test y presento ese resultado.

[3.7858847170108922,
 3.4231937078309103,
 3.973889265795407,
 3.8349755626017683,
 3.821590487864959]